In [1]:
dataset_path=r'/scratch/chil/DSAC-master/7scenes/7scenes_chess/training/scene'

In [164]:
import os
from os import path
import numpy as np
import pdb

In [273]:
img_no='000001479'
depth_path=path.join(dataset_path, 'depth_noseg', img_no+'.png')
pose_path=path.join(dataset_path,'poses',img_no+'.txt')

In [20]:
img_width=640
img_height=480
focalLength = 525.0
xShift = 0.0
yShift = 0.0
secondaryFocalLength = 585.0
rawXShift = 0.0
rawYShift = 0.0
trans = np.array([[0.9998282978278875, 0.008186003849805841, 0.01662420535123559, -0.01393324413905143],
                      [-0.008090415588790156, 0.9999503986429169, -0.005809081637597863, 0.05228905046770047],
                      [-0.01667093393273896, 0.005673587475537798, 0.9998449331606215, 0.02712006871814571],
                      [0, 0, 0, 1]])

In [191]:
def pxToEye(x, y, depth, w, h, cx, cy, f):
    # x.shape=(N)
    eye=np.ones((len(x),4,1))
    zero_index=np.where(depth==0)
    eye[zero_index][:,0:3,0]=0
    
    index=np.where(depth!=0)
    X=((x[index] - (w / 2.0 + cx)) / (f / depth[index]))
    Y=-((y[index] - (h / 2.0 + cy)) / (f / depth[index]))
    Z=-depth[index]
    ones=np.ones_like(X)
    
    coor=np.concatenate((X[np.newaxis],Y[np.newaxis],Z[np.newaxis],ones[np.newaxis])).transpose()
    coor=np.expand_dims(coor,-1)
    eye[index]=coor
    return eye

In [214]:
def eyeToPx(eye, depth, w, h, cx, cy, f):
    #eye.shape=(N,4,1), depth.shape=(N)
    x=np.zeros(len(depth))
    y=np.zeros(len(depth))
    index=np.where(depth!=0)
    # ToDo: replace depth with eye[:,2,0]
    x[index]=(eye[index][:,0,0] * (f / depth[index])) + (w / 2.0 + cx) + 0.5
    y[index]=-(eye[index][:,1,0] * (f / depth[index])) + (h / 2.0 + cy) + 0.5
    return x.astype(np.int),y.astype(np.int)

In [248]:
def mapDepthToRGB(x, y, depth, w, h, dcx, dcy, df, rgbcx, rgbcy, rgbf):
    # project to 3D point in camera coordinates (with intrinsic parameters of the depth sensor)
    eye=pxToEye(x, y, depth, w, h, dcx, dcy, df)
    # apply relative transformation between sensors
    n_trans=np.repeat(trans[np.newaxis],len(eye),axis=0)
    eye=np.matmul(n_trans,eye)
    # project to 2D point in image coordiantes (with intrinsic parameters of the rgb sensor)
    return eyeToPx(eye, depth, w, h, rgbcx, rgbcy, rgbf),eye

In [302]:
def getDepth(depth_img,mode='image'):
    h,w=depth_img.shape
    depth=depth_img.flatten()
    depth=depth.astype(np.float)
    x,y=np.meshgrid(np.arange(w),np.arange(h))
    x=x.flatten()
    y=y.flatten()
    (newx,newy),eye=mapDepthToRGB(x,y,depth,w,h,rawXShift,rawYShift,secondaryFocalLength,xShift,yShift,focalLength)
    dsac_eye=eye[:,0:2,0]
    dsac_eye=np.concatenate([dsac_eye,np.expand_dims(-depth,-1)],axis=1)
    if mode=='image':
        objMap=np.zeros((h,w,3))
        objMap[newy,newx]=dsac_eye
        depthMap=np.zeros((h,w))
        depthMap[newy,newx]=depth
        return depthMap,objMap
    else:
        return newx,newy,dsac_eye

In [294]:
import re
def getInfo(pose_txt):
    a = []
    with open(pose_txt) as f:
        pattern = re.compile('\s+')
        for line in f.readlines():
            ss=re.split(pattern, line.strip())
            for s in ss:
                a.append(float(s))

    a = np.array(a).reshape(4,4)
    ts = [0.6880049706, 0.333539999278, 2.23485151692]
    for i in xrange(3):
        a[i][3] -= ts[i]
        
    correction = np.array([[1,0,0,0],
                            [0,-1,0,0],
                            [0,0,-1,0],
                            [0,0,0,1]])
    a = np.matmul(a,correction)
    for i in xrange(3):
        a[i][3] *= 1000
    return np.linalg.inv(a)

In [304]:
def getObj(depth_img,pose_txt):
    newx,newy,eye = getDepth(depth_img,mode='init')
    ones = np.ones((len(eye),1))
    eye = np.concatenate([eye,ones],axis=1)
    pose = getInfo(pose_txt)
    pose_inv = np.linalg.inv(pose)
    T = np.repeat(pose_inv[np.newaxis],len(newx),axis=0)
    obj = np.matmul(T,np.expand_dims(eye,-1))
    objMap = np.zeros((img_height,img_width,3))
    obj=obj[:,:3,0]
    objMap[newy,newx]=obj
    return objMap

## convert

In [316]:
import cv2
src_dir=r'/scratch/chil/7scenes/chess'
tgt_dir=r'/scratch/chil/7scenes_align/chess'
depth_dirs=['seq-01','seq-02','seq-03','seq-04','seq-05','seq-06']

for depth_dir in depth_dirs:
    if not path.exists(path.join(tgt_dir,depth_dir)):
        os.makedirs(path.join(tgt_dir,depth_dir))
    imgs = os.listdir(path.join(src_dir,depth_dir))
    for i in xrange(1000):
        depth = 'frame-{:0>6d}.depth.png'.format(i)
        pose = 'frame-{:0>6d}.pose.txt'.format(i)
        depth_img=cv2.imread(path.join(src_dir,depth_dir,depth),-1)
        pose_txt=path.join(src_dir,depth_dir,pose)
        objMap=getObj(depth_img,pose_txt)
        tgt_path=path.join(tgt_dir,depth_dir,'{:0>6}.npy'.format(i))
        np.save(tgt_path,objMap)

## fun_test

In [309]:
import cv2
depth_img=cv2.imread(depth_path,-1)
depthMap=getDepth(depth_img)

In [305]:
import cv2
depth_img=cv2.imread(depth_path,-1)
obj=getObj(depth_img,pose_path)

In [308]:
print obj.shape
print obj[107][66]

(480, 640, 3)
[-1193.80173886  -486.05760055    93.76739674]


In [233]:
print depthMap[82][25]

1952.0


In [311]:
np.save('test.npy',obj)

In [312]:
objMap=np.load('test.npy')

In [314]:
objMap[0][0]

array([-1232.05912683,  -792.06021326, -2243.10900513])

In [234]:
depthMap=depthMap.astype(np.uint16)
cv2.imwrite('test.png',depthMap)

True

In [238]:
img=cv2.imread('test.png',-1)
print img.dtype
print img[82][25]

uint16
1952


## Test

In [202]:
import cv2
depth=cv2.imread(depth_path,-1)
#print depth.shape
#print depth
i=200
j=100
test1=depth[i][j]
depth=depth.flatten().astype(np.float)
x,y=np.meshgrid(np.arange(img_width),np.arange(img_height))
x=x.flatten()
y=y.flatten()
no=i*640+j
#print test1,depth[no]
assert test1==depth[no]
eye=pxToEye(x,y,depth,img_width,img_height,xShift,yShift,secondaryFocalLength)
print eye[0]
eye=eye.reshape(480,640,4,1)
print eye[0][0]

[[ -9.89538462e+02]
 [  7.42153846e+02]
 [ -1.80900000e+03]
 [  1.00000000e+00]]
[[ -9.89538462e+02]
 [  7.42153846e+02]
 [ -1.80900000e+03]
 [  1.00000000e+00]]


In [171]:
np.expand_dims(np.arange(3),-1).shape

(3, 1)

In [154]:
x,y=np.meshgrid(np.arange(img_width),np.arange(img_height))
x=x.flatten()
y=y.flatten()
coor=zip(y,x)
print coor[650:690]
print len(coor)
print x.reshape(480,640)
print y.reshape(480,640)

[(1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (1, 24), (1, 25), (1, 26), (1, 27), (1, 28), (1, 29), (1, 30), (1, 31), (1, 32), (1, 33), (1, 34), (1, 35), (1, 36), (1, 37), (1, 38), (1, 39), (1, 40), (1, 41), (1, 42), (1, 43), (1, 44), (1, 45), (1, 46), (1, 47), (1, 48), (1, 49)]
307200
[[  0   1   2 ..., 637 638 639]
 [  0   1   2 ..., 637 638 639]
 [  0   1   2 ..., 637 638 639]
 ..., 
 [  0   1   2 ..., 637 638 639]
 [  0   1   2 ..., 637 638 639]
 [  0   1   2 ..., 637 638 639]]
[[  0   0   0 ...,   0   0   0]
 [  1   1   1 ...,   1   1   1]
 [  2   2   2 ...,   2   2   2]
 ..., 
 [477 477 477 ..., 477 477 477]
 [478 478 478 ..., 478 478 478]
 [479 479 479 ..., 479 479 479]]


In [110]:
a=np.zeros((3,4))
print a
x,y=np.meshgrid(np.arange(4),np.arange(3))
x=x.flatten()
y=y.flatten()
a[y,x]=[5,3,8,9,0,1,3,4,2,5,1,1]
print a

[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
[[ 5.  3.  8.  9.]
 [ 0.  1.  3.  4.]
 [ 2.  5.  1.  1.]]


In [80]:
n_trans=np.repeat(trans[np.newaxis],10,axis=0)
print n_trans.shape
print n_trans[0]
print n_trans[1]

(10, 4, 4)
[[ 0.9998283   0.008186    0.01662421 -0.01393324]
 [-0.00809042  0.9999504  -0.00580908  0.05228905]
 [-0.01667093  0.00567359  0.99984493  0.02712007]
 [ 0.          0.          0.          1.        ]]
[[ 0.9998283   0.008186    0.01662421 -0.01393324]
 [-0.00809042  0.9999504  -0.00580908  0.05228905]
 [-0.01667093  0.00567359  0.99984493  0.02712007]
 [ 0.          0.          0.          1.        ]]


In [61]:
a=np.arange(12).reshape(2,3,2)
b=np.arange(12).reshape(2,2,3)
c=np.matmul(a,b)
print c.shape
print c
print np.dot(a[0],b[0])
print np.dot(a[1],b[1])

(2, 3, 3)
[[[  3   4   5]
  [  9  14  19]
  [ 15  24  33]]

 [[ 99 112 125]
  [129 146 163]
  [159 180 201]]]
[[ 3  4  5]
 [ 9 14 19]
 [15 24 33]]
[[ 99 112 125]
 [129 146 163]
 [159 180 201]]


In [11]:
a=np.arange(10)
print a
index=np.where(a!=0)
print index
print len(index[0])
a[index]=np.arange(9)*2
print a

[0 1 2 3 4 5 6 7 8 9]
(array([1, 2, 3, 4, 5, 6, 7, 8, 9]),)
9
[ 0  0  2  4  6  8 10 12 14 16]


In [84]:
a=np.arange(10)
print a
index=[2,3,2]
a[index]=(9,3,-9)
print a

[0 1 2 3 4 5 6 7 8 9]
[ 0  1 -9  3  4  5  6  7  8  9]


In [21]:
trans

array([[ 0.9998283 ,  0.008186  ,  0.01662421, -0.01393324],
       [-0.00809042,  0.9999504 , -0.00580908,  0.05228905],
       [-0.01667093,  0.00567359,  0.99984493,  0.02712007],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [85]:
a=np.arange(10)+0.5
print a
a=a.astype(np.int)
print a

[ 0.5  1.5  2.5  3.5  4.5  5.5  6.5  7.5  8.5  9.5]
[0 1 2 3 4 5 6 7 8 9]


In [255]:
objs=np.load('obj.npy')